call TrainAndSave.py to train many models with different initialization of quadrupoles

In [2]:
import sys
sys.path.append("../../")

import torch

from cpymad.madx import Madx

load model into Mad-X

In [20]:
output = True
madx = Madx(stdout=False)
madx.options.echo = output 
madx.options.warn = output 
madx.options.info = output 

# specify beam
assert madx.command.beam(mass=18.798, charge=7, exn=1.258e-6, eyn=2.005e-6, gamma=19/18.798)  

# activate sequence
sequence = "SIS18"
madx.call("Lattice/SIS18.seq")
madx.input("""
iqf=3.12391e-01;
iqd=-4.78047e-01;

ik2f=0.0;
ik2d=-0.0;
""")
madx.command.use(sequence=sequence)

True

match workingpoints

In [24]:
hTunes, vTunes = torch.linspace(4.05, 4.45, 10), torch.linspace(3.05, 3.45, 10)

for q1 in hTunes:
    for q2 in vTunes:
        # match tune
        madx.input("""
        match, sequence={};
        global, sequence={}, q1={}, q2={};
        vary, name=iqf, step=0.00001;
        vary, name=iqd, step=0.00001;
        lmdif, calls=500, tolerance=1.0e-10;
        endmatch;
        """.format(sequence, sequence, q1, q2))
        
        iqf, iqd = madx.globals["iqf"], madx.globals["iqd"]
        run = subprocess.Popen(["python", "TrainAndSave.py", str(iqf), str(iqd)],
                               shell=False, stdin=None, stdout=None, stderr=None, close_fds=True)